In [1]:
import pathlib
import json
import re

In [2]:
import influxdb
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from odapi.connectors import Irceline
from odapi.settings import settings
settings.logger.setLevel(50)

## Parameters

In [4]:
t0 = pd.Timestamp('2020-06-22 13:00:00', tz='CET').tz_convert('UTC')
t0

Timestamp('2020-06-22 11:00:00+0000', tz='UTC')

In [5]:
t1 = pd.Timestamp.utcnow()
t1

Timestamp('2020-06-29 14:18:21.459946+0000', tz='UTC')

## Irceline Reference

In [6]:
cref = Irceline()

In [7]:
sel = cref.select(sitekey='41', measurekey='PM')

In [8]:
sel.shape

(11, 25)

In [9]:
recs = cref.get_records(sel, start=t0, stop=t1)

In [10]:
dref = recs.merge(sel[['serieid', 'seriekey']]).pivot_table(index='start', columns='seriekey', values='value')

In [11]:
dref.tail()

seriekey,PM-10.0/41B011 (µg/m³),PM-10.0/41MEU1 (µg/m³),PM-10.0/41N043 (µg/m³),PM-10.0/41R001 (µg/m³),PM-10.0/41R012 (µg/m³),PM-10.0/41WOL1 (µg/m³),PM-2.5/41B011 (µg/m³),PM-2.5/41MEU1 (µg/m³),PM-2.5/41N043 (µg/m³),PM-2.5/41R001 (µg/m³),PM-2.5/41R012 (µg/m³)
start,,,,,,,,,,,
2020-06-29 08:00:00+00:00,9.5,NaN,17.5,18.5,14.5,6.0,1.65,NaN,2.95,5.4,6.35
2020-06-29 09:00:00+00:00,9.5,NaN,NaN,15.0,16.0,NaN,0.85,NaN,NaN,5.4,5.80
2020-06-29 10:00:00+00:00,8.5,NaN,23.0,NaN,14.5,8.5,0.40,NaN,9.65,NaN,4.60
2020-06-29 11:00:00+00:00,NaN,25.5,24.0,NaN,NaN,10.0,NaN,13.15,8.75,NaN,NaN
2020-06-29 12:00:00+00:00,11.5,27.5,26.5,NaN,NaN,9.0,1.10,8.90,8.35,NaN,NaN


## SDS Benchmark

In [12]:
with pathlib.Path('C:/Users/admin/udlog/influx.json').open() as fh:
    client = influxdb.InfluxDBClient(**json.load(fh))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\admin\\udlog\\influx.json'

In [ ]:
client.ping()

In [ ]:
query = """
SELECT mean(pm25) AS pm25, mean(pm10) AS pm10
FROM "home"
WHERE time >= '{start:}' AND time < '{stop:}'
GROUP BY time(1h)
""".format(start=t0.isoformat(), stop=t1.isoformat())
print(query)

In [ ]:
r = client.query(query).raw['series'][0]

In [ ]:
dsds = pd.DataFrame(r['values'], columns=r['columns'])
dsds['time'] = pd.to_datetime(dsds['time'])
dsds = dsds.set_index('time')
dsds.columns = ['PM-2.5/Home (µg/m³)', 'PM-10.0/Home (µg/m³)']
dsds.tail()

## Data Merge

In [ ]:
df = pd.concat([dref, dsds], axis=1)
df.tail()

## Render

In [ ]:
remeas = re.compile('([\w\-\.]{2,8})/.*(\(.*\))')
resite = re.compile('/([\w]{4,6}) ')

In [ ]:
fig, axe = plt.subplots()
axe = df.filter(regex='2.5').plot(ax=axe)
axe.legend(bbox_to_anchor=(1,1))
axe.set_ylim([0, 100])
axe.grid()
labels = [s.get_text() for s in axe.get_legend().get_texts()]
axe.set_title("Concentration Time Series on [{:%Y-%m-%d}; {:%Y-%m-%d}]".format(t0, t1))
axe.set_xlabel("Time")
axe.set_ylabel('{} {}'.format(*remeas.search(labels[0]).groups()))
_ = fig.text(0.92, 0.15, "Sources: Irceline API\nCredits: J. Landercy", rotation=90)

In [ ]:
fig, axe = plt.subplots()
axe = df.filter(regex='10.0').plot(ax=axe)
axe.legend(bbox_to_anchor=(1,1))
axe.set_ylim([0, 100])
axe.grid()
labels = [s.get_text() for s in axe.get_legend().get_texts()]
axe.set_title("Concentration Time Series on [{:%Y-%m-%d}; {:%Y-%m-%d}]".format(t0, t1))
axe.set_xlabel("Time")
axe.set_ylabel('{} {}'.format(*remeas.search(labels[0]).groups()))
_ = fig.text(0.92, 0.15, "Sources: Irceline API\nCredits: J. Landercy", rotation=90)

In [ ]:
fig, axe = plt.subplots()
axe = df.filter(regex='10.0').boxplot(rot=90, ax=axe)
axe.set_ylim([0, 50])
labels = [s.get_text() for s in axe.get_xticklabels()]
axe.set_xticklabels([resite.search(s).group(1) for s in labels])
axe.set_title("Concentration Distributions on [{:%Y-%m-%d}; {:%Y-%m-%d}]".format(t0, t1))
axe.set_xlabel("Sites")
axe.set_ylabel('{} {}'.format(*remeas.search(labels[0]).groups()))
_ = fig.text(0.92, 0.15, "Sources: Irceline API\nCredits: J. Landercy", rotation=90)

In [ ]:
fig, axe = plt.subplots()
axe = df.filter(regex='2.5').boxplot(rot=90, ax=axe)
axe.set_ylim([0, 50])
labels = [s.get_text() for s in axe.get_xticklabels()]
axe.set_xticklabels([resite.search(s).group(1) for s in labels])
axe.set_title("Concentration Distributions on [{:%Y-%m-%d}; {:%Y-%m-%d}]".format(t0, t1))
axe.set_xlabel("Sites")
axe.set_ylabel('{} {}'.format(*remeas.search(labels[0]).groups()))
_ = fig.text(0.92, 0.15, "Sources: Irceline API\nCredits: J. Landercy", rotation=90)